In [5]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt",use_fast = False, src_lang="ja_XX", tgt_lang="en_XX")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
article_jp = "【起】［このカードを【レスト】する］ あなたは自分のレベル1以上のキャラを1枚選び、そのターン中、パワーを＋1500。【起】 集中 ［(1) このカードを【レスト】する］ あなたは自分の山札の上から4枚をめくり、控え室に置く。それらのカードのクライマックス1枚につき、あなたは自分の控え室の《ホロライブ》のキャラを1枚まで選び、手札に戻す。"
tokenizer.src_lang = "ja_XX"
encoded_ar = tokenizer(article_jp, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)



['[Start][Restore this card] You select one of your Level 1 or higher characters, and during that turn, you will have the power of +1500.[Start] Concentration [(1) Restore this card] You pick up four cards from your mountain札 and place them in the holding room.For each of those cards\' climax, you select one of your holding room\'s "Holo Live" characters and put them back in your hand.']

In [1]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50Tokenizer


model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="ja_XX", tgt_lang="en_XX")
dataset = load_dataset('json', data_files={'train': 'train.json', 'validation': 'validation.json'})

def preprocess_function(examples):
    inputs = examples['ja']
    targets = examples['en']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    labels = tokenizer(text_target=targets, max_length=128, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

print(tokenized_datasets['train'][0])
print(tokenized_datasets['validation'][0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

{'en': '[S] [Rest this] Choose 1 of your Level 1 or higher Characters, and for the turn that Character gains +1500 Power.[S] BRAINSTORM [(1) Rest this] Flip over the top 4 cards of your Library and put them in the Waiting Room. For each Climax card revealed this way, choose up to 1 ::Hololive:: Character in your Waiting Room and return it to your hand.', 'ja': '【起】［このカードを【レスト】する］ あなたは自分のレベル1以上のキャラを1枚選び、そのターン中、パワーを＋1500。【起】 集中 ［(1) このカードを【レスト】する］ あなたは自分の山札の上から4枚をめくり、控え室に置く。それらのカードのクライマックス1枚につき、あなたは自分の控え室の《ホロライブ》のキャラを1枚まで選び、手札に戻す。', 'input_ids': [250012, 5946, 2797, 2728, 1065, 3619, 38268, 251, 7662, 11375, 17444, 2728, 1786, 268, 6, 136602, 11748, 79503, 418, 59460, 194190, 251, 418, 50305, 87969, 37, 3230, 17852, 7740, 514, 37, 189447, 251, 21748, 4283, 145674, 2797, 2728, 6, 30792, 378, 27750, 11207, 38268, 251, 7662, 11375, 17444, 2728, 1786, 268, 6, 136602, 11748, 2272, 146420, 81045, 1309, 617, 50305, 251, 8421, 2928, 2694, 37, 210852, 8481, 327, 15760, 2928, 30, 147851, 154, 3826

In [2]:
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="ja_XX", tgt_lang="en_XX")
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
)

# Initialize the Trainer and Start Training
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

model.save_pretrained('./finetuned_model')
tokenizer.save_pretrained('./finetuned_model')



Epoch,Training Loss,Validation Loss
1,No log,1.649812
2,No log,1.336024
3,No log,1.197812


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


In [3]:
model = MBartForConditionalGeneration.from_pretrained('./finetuned_model')
tokenizer = MBart50Tokenizer.from_pretrained('./finetuned_model', src_lang="ja_XX", tgt_lang="en_XX")
article_jp = "【起】［このカードを【レスト】する］ あなたは自分のレベル1以上のキャラを1枚選び、そのターン中、パワーを＋1500。【起】 集中 ［(1) このカードを【レスト】する］ あなたは自分の山札の上から4枚をめくり、控え室に置く。それらのカードのクライマックス1枚につき、あなたは自分の控え室の《ホロライブ》のキャラを1枚まで選び、手札に戻す。"
tokenizer.src_lang = "ja_XX"
encoded_ar = tokenizer(article_jp, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['[Start] [Restore] Select one of your Level 1 or higher Characters and gain +1500 Power during that turn.[Start] Focus [(1) Restore] Move 4 cards from your Mountain Cards and put them in the Warning Room.For each one of the Climax Characters, select up to 1 Character of "Hololive" in your Warning Room and return it to your hand.']